In [15]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Choose the ratio you want between the training set and test set
desired_test_ratio = 0.2  # For example, 20% for the test set

# Calculate the number of samples to move to the test set
num_samples_to_move = int(x_train.shape[0] * desired_test_ratio)

# Move samples from the training set to the test set
x_test = np.concatenate((x_test, x_train[:num_samples_to_move]), axis=0)
y_test = np.concatenate((y_test, y_train[:num_samples_to_move]), axis=0)

# Update the training set by removing the selected samples and labels
x_train = x_train[num_samples_to_move:]
y_train = y_train[num_samples_to_move:]

# Preprocess the data
x_train = x_train.reshape(-1, 784) / 255.0
x_test = x_test.reshape(-1, 784) / 255.0

# Convert the labels to one-hot encoded vectors
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

# Build the neural network model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(784,)))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define the ranges for pred_prob
ranges = [(0,0.1),(0.1, 0.2), (0.2,0.3), (0.3, 0.4),(0.4,0.5), (0.5, 0.6),(0.6,0.7),(0.7,0.8),(0.8,0.9),(0.9,1)]


# ranges = [(0.9,1)]



In [16]:

num_images_per_range = 50
model.fit(x_train, y_train, epochs=5, batch_size=32, verbose=1)
predictions = model.predict(x_test)
# Find the indices of test images within each pred_prob range
selected_indices = []
index_num = 9
for start, end in ranges:
    range_indices = [i for i, label in enumerate(y_test) if (start < predictions[i][index_num] < end)]
    selected_indices.extend(range_indices[:num_images_per_range])

Epoch 1/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3412 - accuracy: 0.9045
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.1677 - accuracy: 0.9514
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.1217 - accuracy: 0.9636
Epoch 4/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.0952 - accuracy: 0.9715
Epoch 5/5
688/688 [==============================] - 2s 2ms/step


In [17]:
import pandas as pd
print(selected_indices)
pd.DataFrame(selected_indices).to_csv("selected_indices.csv")

[0, 1, 2, 3, 4, 5, 6, 8, 10, 11, 13, 14, 15, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 149, 227, 250, 376, 571, 950, 1096, 1216, 1516, 1575, 1607, 1701, 1800, 1952, 1956, 1981, 2292, 2369, 2408, 2607, 2630, 2645, 2963, 3005, 3055, 3172, 3225, 3232, 3410, 3429, 3475, 3571, 3653, 3730, 3769, 3793, 4007, 4194, 4266, 4300, 4335, 4356, 4363, 4519, 4695, 4721, 4893, 5617, 5709, 5835, 116, 166, 241, 448, 460, 1422, 1494, 1614, 2028, 2334, 2406, 2530, 3060, 4297, 4355, 4598, 4740, 4783, 4860, 4966, 5201, 5440, 5734, 6560, 6571, 7350, 8061, 8527, 9208, 10053, 10718, 11024, 11446, 11968, 12050, 12208, 12490, 12546, 13828, 14346, 14802, 14945, 15174, 15324, 15338, 15372, 15408, 16160, 16534, 16711, 628, 959, 1062, 1301, 1440, 1453, 1553, 1581, 1816, 1865, 2189, 2683, 2907, 2931, 3282, 3437, 4078, 4294, 4379, 4500, 4827, 4998, 5525, 6045, 6564, 6570, 9536, 9764, 10420, 10754, 11522, 11744, 118

In [28]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Build the neural network model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(784,)))
model.add(Dense(64, activation='relu', input_shape=(784,)))

model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model for a total of 100 epochs, making predictions every epoch
total_epochs = 50

epoch_loss = []
epoch_accuracy = []

for epoch in range(1, total_epochs + 1):
    # Train for one epoch
    model.fit(x_train, y_train, epochs=1, batch_size=32, verbose=1)

    # Evaluate the model on the test set
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    epoch_loss.append(loss)
    epoch_accuracy.append(accuracy)
    
    print(f'Epoch {epoch}/{total_epochs} - Test loss: {loss:.4f} - Test accuracy: {accuracy:.4f}')
    # mdf = pd.DataFrame()

    # Your predictions and DataFrame code here
    predictions = model.predict(x_test)
    df_list = []

    for i, index in enumerate(selected_indices, start=1):
        prediction = predictions[index]
        image = x_test[index].reshape(28, 28)
        pred_prob = prediction[index_num]
        label = y_test[index]
        df_list.append({"pred_prob": pred_prob, "is_nine": (label[index_num] == 1)})

    # mdf = pd.concat([mdf,pd.DataFrame(df_list)], ignore_index=True)

    pd.DataFrame(df_list).to_csv(f"Epoch-{epoch}-result.csv")

# Convert lists to DataFrames with index
epoch_loss_df = pd.DataFrame({'Loss': epoch_loss, 'Epoch': range(1, total_epochs + 1)})
epoch_accuracy_df = pd.DataFrame({'Accuracy': epoch_accuracy, 'Epoch': range(1, total_epochs + 1)})

# Save DataFrames to CSV
epoch_loss_df.to_csv('epoch_loss.csv', index=False)
epoch_accuracy_df.to_csv('epoch_accuracy.csv', index=False)



1500/1500 [==============================] - 5s 3ms/step - loss: 0.3095 - accuracy: 0.9102
Epoch 1/50 - Test loss: 0.1700 - Test accuracy: 0.9517
1500/1500 [==============================] - 4s 3ms/step - loss: 0.1448 - accuracy: 0.9564
Epoch 2/50 - Test loss: 0.1227 - Test accuracy: 0.9642
1500/1500 [==============================] - 5s 3ms/step - loss: 0.1071 - accuracy: 0.9672
Epoch 3/50 - Test loss: 0.1185 - Test accuracy: 0.9641
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0837 - accuracy: 0.9740
Epoch 4/50 - Test loss: 0.1005 - Test accuracy: 0.9703
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0689 - accuracy: 0.9779
Epoch 5/50 - Test loss: 0.0931 - Test accuracy: 0.9716
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0565 - accuracy: 0.9811
Epoch 6/50 - Test loss: 0.1023 - Test accuracy: 0.9699
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0477 - accuracy: 0.9843
Epoch 7/50 - Test loss: 0.0993 - 

[]
